# In this notebook I write the routine for TOF spectrum calibration

        What it should be done:
        - read the theoretical spectrum
        - read the calibration sample
        - be able to convert from TOF to lambda and back
        - calculate T0 and deltaL, which are the results of the calibration 
        
        time to lambda convertion:
            lambda = h/mL (t-t0)
            where:lambda = wavelength [A] (A=0.1nm = 1e-10m)
                    h = Planck's constant: 6.62607004 × 10-34 m^2 kg / s
                    m = Neutron mass [kg]: 1.674 927 471 x 10-27 kg 
                    L = total flight path [m]
                    t = time of flight [s]
                    
        

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
import os, fnmatch
from os import listdir
%matplotlib inline
import scipy.signal
print(scipy.__version__)
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
import AdvancedBraggEdgeFitting


/home/carminati_c/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


1.2.0


In [2]:
# Constant values
h=6.62607004e-34 #Planck constant [m^2 kg / s]
m=1.674927471e-27 #Neutron mass [kg]
t0=0
L= 56.4 #[m]
dL = 0

In [ ]:
def tof2l(tof):
    l=h/m*(tof-t0)/(L+dL)/1e-10
    return l

def l2tof(l):
    tof=t0+(l*1e-10)*(L+dL)*m/h
    return tof

In [ ]:
mylambda = np.genfromtxt('lambda.txt',usecols=0)
myspectrum = np.genfromtxt('alpha.txt',usecols=0)

# print(mylambda)
mytof = l2tof(mylambda)
relambda = tof2l(mytof)
plt.plot(mylambda, myspectrum/max(myspectrum))
plt.show()


In [ ]:
# Read the calibration spectrum from file
mycaltof = np.genfromtxt('/media/carminati_c/Elements/RB1820231/02_HighStats_Radio_1hruns/samples_after_reboot_OC_Fiji/IMAT00010420_HighStats_Radio_1hruns_000_Spectra.txt', usecols=0)
myhist = np.genfromtxt('/media/carminati_c/Elements/RB1820231/02_HighStats_Radio_1hruns/samples_after_reboot_OC_Fiji/IMAT00010420_HighStats_Radio_1hruns_000_Spectra.txt', usecols=1) #this is the cumulative histogram of the raw data (before the overlap correction)

plt.plot(mycaltof,myhist)
plt.show()

In [ ]:
# Read the calibration datasets:
pathdata ="/media/carminati_c/Elements/RB1820231/02_HighStats_Radio_1hruns/samples_after_reboot_OC_Fiji/"
pathOB = "/media/carminati_c/Elements/RB1820231/02_HighStats_Radio_1hruns/flat_after_reboot_OC_Fiji/"

In [ ]:
myfiles = fnmatch.filter(listdir(pathdata),'*.fits')
coll_files = sorted(myfiles)
print(coll_files[504])
print(coll_files[505])
print(coll_files[506])

obfiles = fnmatch.filter(listdir(pathOB),'*.fits') # here there are several OB folders
coll_ob = sorted(obfiles)
print(coll_ob[504])
print(coll_ob[505])
print(coll_ob[506])
# print(sorted(obfiles))

In [ ]:
# here I don't have repeated measurement for the OB for the cal sample

# # normalized_sample = np.zeros([512,512,len(coll_files)])
# obsubfiles = [None]*len(obfiles)

# for i in range(0, len(obfiles)):
# #     print(obfiles[i])
#     obsubfiles[i]= sorted(fnmatch.filter(listdir(pathOB+obfiles[i]+'/Corrected'),'*.fits')) 

# print(obsubfiles[2][0])

In [ ]:
roi_cal = np.array([6,16,57,500])
filename = pathdata + coll_files[0]
im = fits.open(filename)
# plt.imshow(im[0].data)
plt.imshow(im[0].data[roi_cal[1]:roi_cal[3],roi_cal[0]:roi_cal[2]]) #this is the area that I want to study

In [ ]:
cal_spectrum = np.zeros(len(coll_files))
cal_ob = np.zeros(len(coll_files))
ori_hist = np.zeros(len(coll_files))

for i in range(0, len(coll_files)):
    
    curr_img = (fits.open(pathdata+coll_files[i])[0].data[roi_cal[1]:roi_cal[3],roi_cal[0]:roi_cal[2]]).astype(float)
    curr_ob =(fits.open(pathOB+coll_ob[i])[0].data[roi_cal[1]:roi_cal[3],roi_cal[0]:roi_cal[2]]).astype(float)
    cal_spectrum[i] = np.sum(curr_img[~np.isnan(curr_img) & ~np.isinf(curr_img)])
    cal_ob[i]= np.sum(curr_ob[~np.isnan(curr_ob) & ~np.isinf(curr_ob)])
    
    

In [ ]:
cal_spectrum_norm = -1*np.log(cal_spectrum/cal_ob)
mycalLambda = tof2l(mycaltof)
plt.plot(mycalLambda, -1*np.log(cal_spectrum/cal_ob))
plt.xlim(0,8)
plt.show()

myspectrum_norm = myspectrum/np.average(myspectrum)
plt.plot(mylambda, myspectrum)
plt.xlim(0,8)
plt.show()

In [ ]:

# yhat = savgol_filter(y, 51, 3) # window size 51, polynomial order 3
peaks, _ = find_peaks(cal_spectrum_norm, width=15)
plt.plot(mycalLambda, cal_spectrum_norm)
plt.plot(mycalLambda[peaks],cal_spectrum_norm[peaks],'x', markeredgewidth=3)
# plt.ylim(0.8,1.02)
plt.show()
print(mycalLambda[peaks])

peaks_th, _ = find_peaks(myspectrum_norm, width=2)
plt.plot(mylambda, myspectrum_norm)
plt.plot(mylambda[peaks_th],myspectrum_norm[peaks_th],'x', markeredgewidth=3, c='orange')
# plt.ylim(0.3,1.02)
plt.show()
print(mylambda[peaks_th])

In [ ]:
#Here I try to use the BE Advanced fitting to fit single edges
print(peaks) # here I see which are the positions of the peaks
# plt.plot(mycalLambda[1000:], cal_spectrum_norm[1000:])
myLastPeak = cal_spectrum_norm[1000:]
print(peaks[-1])# position of the last peak
# plt.plot(mycalLambda[1000:],myLastPeak)
myrange = np.array([1000, len(cal_spectrum)-1])
est_sigma=-1
est_alpha=-10

AdvancedBraggEdgeFitting.AdvancedBraggEdgeFitting(cal_spectrum_norm, myrange, peaks[-1], est_sigma, est_alpha)


In [ ]:
# def term0(t,a2,a6):
#     return  a2 * (t - a6)

# def term1(t,a2,a5,a6):
#     return ((a5 - a2) / 2) * (t - a6)

# def term3(t,t0,sigma):
#     return erfc(-((t-t0)/(sigma * math.sqrt(2))))

# def term4(t,t0,alpha,sigma):
#     return np.exp(-((t-t0)/alpha) + ((sigma*sigma)/(2*alpha*alpha)))

# def term5(t,t0,alpha,sigma):
#     return erfc(-((t-t0)/(sigma * math.sqrt(2))) + sigma/alpha)

# #TO DO: add the option to fitt either shape (?) or not
# def BraggEdgeShape(t,t0,alpha,sigma,a1,a2,a5,a6):
#     return a1 + term0(t,a2,a6) + term1(t,a2,a5,a6) * (1-(term3(t,t0,sigma) - term4(t,t0,alpha,sigma)* term5(t,t0,alpha,sigma)))

# def BraggEdgeShape2(t,t0,alpha,sigma,a1,a2,a5,a6):
#     return a1 + term0(t,a2,a6) + term1(t,a2,a5,a6) * ((term3(t,t0,sigma) - term4(t,t0,alpha,sigma)* term5(t,t0,alpha,sigma)))

# #I am not sure that all these steps are necessary, most likely one can also only define one and decide how many to fit: TODO understand this 
# def AdvancedBraggEdgeFittingFirstStep(t,a1,a6):
#     return a1 + term0(t,a2_f,a6) + term1(t,a2_f,a5_f,a6) * (1-(term3(t,t0_f,sigma_f) - term4(t,t0_f,alpha_f,sigma_f)* term5(t,t0_f,alpha_f,sigma_f)))

# def AdvancedBraggEdgeFittingSecondStep(t,a2,a5):
#     return a1_f + term0(t,a2,a6_f) + term1(t,a2,a5,a6_f) * (1-(term3(t,t0_f,sigma_f) - term4(t,t0_f,alpha_f,sigma_f)* term5(t,t0_f,alpha_f,sigma_f)))

# def AdvancedBraggEdegFittingThirdStep(t,t0,sigma,alpha):
#     return a1_f + term0(t,a2_f,a6_f) + term1(t,a2_f,a5_f,a6_f) * (1-(term3(t,t0,sigma) - term4(t,t0,alpha,sigma)* term5(t,t0,alpha,sigma)))

# def AdvancedBraggEdegFittingFourthStep(t,a1,a2,a5,a6):
#     return a1 + term0(t,a2,a6) + term1(t,a2,a5,a6) * (1-(term3(t,t0_f,sigma_f) - term4(t,t0_f,alpha_f,sigma_f)* term5(t,t0_f,alpha_f,sigma_f)))

# def AdvancedBraggEdegFittingFifthStep(t,t0,sigma,alpha): #this is just the same as the third
#     return a1_f + term0(t,a2_f,a6_f) + term1(t,a2_f,a5_f,a6_f) * (1-(term3(t,t0,sigma) - term4(t,t0,alpha,sigma)* term5(t,t0,alpha,sigma)))

# def AdvancedBraggEdegFittingAll(t,t0,sigma,alpha,a1,a2,a5,a6):
#     return a1 + term0(t,a2,a6) + term1(t,a2,a5,a6) * (1-(term3(t,t0,sigma) - term4(t,t0,alpha,sigma)* term5(t,t0,alpha,sigma)))


In [ ]:


# def AdvancedBraggEdgeFitting(myspectrum, myrange, est_pos, est_sigma, est_alpha):
    
#     #get the part of the spectrum that I want to fit
# #     mybragg = -1*np.log(myspectrum[myrange[0]:myrange[1]]/np.max(myspectrum[myrange[0]:myrange[1]]))
# #     mybragg = mybragg/np.max(mybragg)# iniziamo senza rumore aggiunto
#     mybragg= myspectrum[myrange[0]:myrange[1]]
#     plt.figure
#     plt.plot(mybragg)
    
#     #first step: estimate the linear function before and after the Bragg Edge
    
#     est_pos=est_pos-myrange[0]
#     t=np.linspace(0,np.size(mybragg)-1,np.size(mybragg))
#     t_before= t[0:est_pos-int(est_pos*0.1)]
#     bragg_before=mybragg[0:est_pos-int(est_pos*0.1)]
#     t_after= t[est_pos+int(est_pos*0.1):-1]
#     bragg_after=mybragg[est_pos+int(est_pos*0.1):-1]
    
#     [slope_before, interception_before] = np.polyfit(t_before, bragg_before, 1)
#     [slope_after, interception_after] = np.polyfit(t_after, bragg_after, 1)
    
#     plt.figure
#     plt.plot(t_before,bragg_before,'.k')
#     plt.plot(t_after,bragg_after,'.k')
#     plt.plot(t,mybragg)

#     plt.plot(t,interception_before+slope_before*t)
#     plt.plot(t,interception_after+slope_after*t)
    
#     #first guess of paramters
#     t0_f=est_pos
#     a2_f=slope_before
#     a5_f=slope_after
#     a6_f=t0_f-(2*mybragg[est_pos+int(est_pos*0.25)]/(a5_f-a2_f)) # I assume edge intensity is equal to 1 as I am normalizing THAT IS NOT ALWAYS THE CASE 
#     a1_f=interception_before+a2_f*a6_f
# #     sigma_f=-1
# #     alpha_f=-10

#     sigma_f = est_sigma
#     alpha_f = est_alpha
#     # method='trust_exact'
#     # method='nelder' #not bad
#     # method='differential_evolution' # needs bounds
#     # method='basinhopping' # not bad
#     # method='lmsquare' # this should implement the Levemberq-Marquardt but it says Nelder-Mead method (which should be Amoeba)
#     method ='least_squares' # default and it implements the Levenberg-Marquardt
    

#     gmodel = Model(AdvancedBraggEdegFittingAll)
    
#     params = gmodel.make_params(t0=t0_f,sigma=sigma_f, alpha=alpha_f, a1=a1_f, a2=a2_f, a5=a5_f, a6=a6_f)
#     params['alpha'].vary = False
#     params['sigma'].vary = False
#     params['t0'].vary = False
#     params['a2'].vary= False
#     params['a5'].vary = False
    
    
#     result1 = gmodel.fit(mybragg, params, t=t, method=method, nan_policy='propagate')
#     print(result1.fit_report())
    
#     plt.figure
#     plt.plot(t, mybragg, 'b-')
#     plt.plot(t, result1.init_fit, 'k--')
#     plt.plot(t, result1.best_fit, 'r-')
#     plt.show()
    
#     a1_f=result1.best_values.get('a1')
#     a6_f=result1.best_values.get('a6')
    
#     params = gmodel.make_params(t0=t0_f,sigma=sigma_f, alpha=alpha_f, a1=a1_f, a2=a2_f, a5=a5_f, a6=a6_f)
#     params['alpha'].vary = False
#     params['sigma'].vary = False
#     params['t0'].vary = False
#     params['a1'].vary= False
#     params['a6'].vary = False
        

#     result2 = gmodel.fit(mybragg, params, t=t, method=method, nan_policy='propagate')
#     print(result2.fit_report())
#     plt.figure
#     plt.plot(t, mybragg, 'b-')
#     plt.plot(t, result2.init_fit, 'k--')
#     plt.plot(t, result2.best_fit, 'r-')
#     plt.show()

#     a2_f = result2.best_values.get('a2')
#     a5_f = result2.best_values.get('a5')
    
#     params = gmodel.make_params(t0=t0_f,sigma=sigma_f, alpha=alpha_f, a1=a1_f, a2=a2_f, a5=a5_f, a6=a6_f)
#     params['a2'].vary = False
#     params['a5'].vary = False
#     params['a1'].vary= False
#     params['a6'].vary = False
#     params['sigma'].vary= False
#     params['alpha'].vary = False


#     result3=gmodel.fit(mybragg, params, t=t, method=method, nan_policy='propagate')
#     print(result3.fit_report())
#     plt.figure
#     plt.plot(t, mybragg, 'b-')
#     plt.plot(t, result3.init_fit, 'k--')
#     plt.plot(t, result3.best_fit, 'r-')
#     plt.show()
    
#     t0_f=result3.best_values.get('t0')
# #     sigma_f=result3.best_values.get('sigma')
# #     alpha_f=result3.best_values.get('alpha')

#     params = gmodel.make_params(t0=t0_f,sigma=sigma_f, alpha=alpha_f, a1=a1_f, a2=a2_f, a5=a5_f, a6=a6_f)
#     params['a2'].vary = False
#     params['a5'].vary = False
#     params['a1'].vary= False
#     params['a6'].vary = False
# #     params['t0'].vary= False

#     result4=gmodel.fit(mybragg, params, t=t, nan_policy='propagate',method=method)
                       


#     print(result4.fit_report())
#     plt.figure
#     plt.plot(t, mybragg, 'b-')
#     plt.plot(t, result4.init_fit, 'k--')
#     plt.plot(t, result4.best_fit, 'r-')
#     plt.show()
    
#     sigma_f=result4.best_values.get('sigma')
#     alpha_f=result4.best_values.get('alpha')
#     t0_f=result4.best_values.get('t0')
    
    
#     params = gmodel.make_params(t0=t0_f,sigma=sigma_f, alpha=alpha_f, a1=a1_f, a2=a2_f, a5=a5_f, a6=a6_f)
#     params['t0'].vary = False
#     params['sigma'].vary = False
#     params['alpha'].vary= False
    
    
#     result5 = gmodel.fit(mybragg, params, t=t, nan_policy='propagate', method=method)
#     print(result5.fit_report())
#     plt.figure
#     plt.plot(t, mybragg, 'b-')
#     plt.plot(t, result5.init_fit, 'k--')
#     plt.plot(t, result5.best_fit, 'r-')
#     plt.show()
    
#     a1_f =result5.best_values.get('a1')
#     a2_f = result5.best_values.get('a2')
#     a5_f = result5.best_values.get('a5')
#     a6_f = result5.best_values.get('a6')

# #     t0_f=result5.best_values.get('t0')
# #     sigma_f=result5.best_values.get('sigma')
# #     alpha_f=result5.best_values.get('alpha')

#     params = gmodel.make_params(t0=t0_f,sigma=sigma_f, alpha=alpha_f, a1=a1_f, a2=a2_f, a5=a5_f, a6=a6_f)
#     params['a2'].vary = False
#     params['a5'].vary = False
#     params['a1'].vary= False
#     params['a6'].vary = False
    
#     result6= gmodel.fit(mybragg, params, t=t, nan_policy='propagate', method=method)



# #     result6=gmodel6.fit(mybragg,t=t, t0=t0_f,sigma=sigma_f, alpha=alpha_f, a1=a1_f, a2=a2_f, a5=a5_f, a6=a6_f, nan_policy='propagate', method=method)
    
#     print(result6.fit_report())
#     plt.figure
#     plt.plot(t, mybragg, 'b-')
#     plt.plot(t, result6.init_fit, 'k--')
#     plt.plot(t, result6.best_fit, 'r-')
#     plt.show()

#     t0_f=result6.best_values.get('t0')
#     sigma_f=result6.best_values.get('sigma')
#     alpha_f=result6.best_values.get('alpha')

#     params = gmodel.make_params(t0=t0_f,sigma=sigma_f, alpha=alpha_f, a1=a1_f, a2=a2_f, a5=a5_f, a6=a6_f)
#     result7 = gmodel.fit(mybragg, params, t=t, nan_policy='propagate', method=method)
    
#     print(result7.fit_report())
#     plt.figure
#     plt.plot(t, mybragg, 'b-')
#     plt.plot(t, result7.init_fit, 'k--')
#     plt.plot(t, result7.best_fit, 'r-')
#     plt.plot(t[t0_f], mybragg[t0_f],'ok')
#     plt.show()
    
#     t0_f=result7.best_values.get('t0')
#     sigma_f=result7.best_values.get('sigma')
#     alpha_f=result7.best_values.get('alpha')
#     a1_f =result7.best_values.get('a1')
#     a2_f = result7.best_values.get('a2')
#     a5_f = result7.best_values.get('a5')
#     a6_f = result7.best_values.get('a6')
    
    
    

## I will then fit the calculated TOF to the theoretical lambda, x= mylambda y=mycaltof 

In [ ]:
z = np.polyfit(mylambda[peaks_th[3:]],mycaltof[peaks[:]],1) # here I have to select which peaks to be used
print(z)
print(mycalLambda[peaks[:]])
print(mylambda[peaks_th[3:]])


plt.plot( mylambda[peaks_th[3:]],mycaltof[peaks[0:6]],'ob')
plt.plot( mylambda, mytof,'-r')
plt.plot(mylambda, mylambda*z[0]+z[1],'-g')
plt.xlim(1,5)
plt.xlabel('wavelength theoretical [A]')
plt.ylabel('ToF measured [s]')
# plt.ylim(0.0,0.03)
plt.legend(['calibration points','theoretical curve', 'fit'])
plt.title('Calibration plot')


In [ ]:
T0 = z[1]
L = (z[0]*h/m)/1e-10
print(T0,L)